# Model 1 Logistic Regression

In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

## Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("Resources/exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


## Select your features (columns)

In [5]:
df.columns

Index(['koi_disposition', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2',
       'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec',
       'koi_kepmag'],
      dtype='object')

In [6]:
# Get rid of unnecessary columns (columns that are not independently measured)
# Set features. This will also be used as your x values.
# Assign X (data) and y (target)

X = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec', 
         'koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 
         'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol', 
         'koi_model_snr', 'koi_steff', 'koi_slogg', 'koi_srad', 
         'ra', 'dec', 'koi_kepmag']]
feature_names = X.columns

y = df['koi_disposition']

print(X.shape, y.shape)

(6991, 19) (6991,)


## Create a Train Test Split

Use `koi_disposition` for the y values

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [8]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
6122,0,0,0,0,6.768901,133.077240,0.150,3.61600,123.1,1.24,1017,253.30,10.8,5737,4.327,1.125,294.40472,39.351681,14.725
6370,0,1,0,1,0.733726,132.020050,0.291,2.30900,114.6,0.86,1867,2891.64,13.8,5855,4.578,0.797,284.50391,42.463860,15.770
2879,1,0,0,0,7.652707,134.460380,0.970,79.89690,641.1,3.21,989,226.81,254.3,6328,4.481,0.963,295.50211,38.983540,13.099
107,0,0,0,0,7.953547,174.662240,0.300,2.63120,875.4,2.25,696,55.37,38.4,4768,4.536,0.779,291.15878,40.750271,15.660
29,0,0,0,0,4.959319,172.258529,0.831,2.22739,9802.0,12.21,1103,349.40,696.5,5712,4.359,1.082,292.16705,48.727589,15.263


## Pre-processing

Scale the data using the MinMaxScaler and encode y 

In [9]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

## Create and Fit the Model

In [10]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=400)
model.fit(X_train_scaled, encoded_y_train)

LogisticRegression(max_iter=400)

In [11]:
# get importance
importance = model.coef_[0]
impt = []
# summarize feature importance
for i,v in enumerate(importance):
    impt.append(v)
sorted(zip(impt, X.columns), reverse=True)

[(1.958257835060407, 'koi_period'),
 (1.540390215928863, 'koi_teq'),
 (1.0224962884956725, 'koi_steff'),
 (0.7399855200669825, 'koi_kepmag'),
 (0.26246151310040844, 'koi_srad'),
 (0.24884200133506224, 'koi_impact'),
 (0.013424671764430973, 'ra'),
 (0.005121513293016564, 'koi_insol'),
 (-0.021980462490128402, 'koi_prad'),
 (-0.29982402711266704, 'dec'),
 (-0.323150579714172, 'koi_depth'),
 (-0.9083390133710603, 'koi_time0bk'),
 (-1.0152564741335015, 'koi_slogg'),
 (-1.2885940416413404, 'koi_duration'),
 (-2.351537840563986, 'koi_fpflag_ss'),
 (-2.4876680949749357, 'koi_fpflag_ec'),
 (-2.828942264964922, 'koi_model_snr'),
 (-3.110771771348206, 'koi_fpflag_co'),
 (-3.6807594328285647, 'koi_fpflag_nt')]

In [12]:
# Set significant features - those furthest from 0

X = X[['koi_period', 'koi_teq', 'koi_steff', 'koi_kepmag', 
       'koi_srad', 'koi_fpflag_ss', 'koi_fpflag_ec', 'koi_model_snr', 
       'koi_fpflag_co', 'koi_fpflag_nt']]
           
feature_names = X.columns
X.head()

,koi_period,koi_teq,koi_steff,koi_kepmag,koi_srad,koi_fpflag_ss,koi_fpflag_ec,koi_model_snr,koi_fpflag_co,koi_fpflag_nt
0,54.418383,443,5455,15.347,0.927,0,0,25.8,0,0
1,19.899140,638,5853,15.436,0.868,1,0,76.3,0,0
2,1.736952,1395,5805,15.597,0.791,1,0,505.6,0,0
3,2.525592,1406,6031,15.509,1.046,0,0,40.9,0,0
4,4.134435,1160,6046,15.714,0.972,0,0,40.2,0,0


## Train the Model

In [13]:
# Recreate train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Rescale data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Retrain model
model = LogisticRegression(max_iter=400)
model.fit(X_train_scaled, encoded_y_train)

LogisticRegression(max_iter=400)

In [14]:
print(f"Training Data Score: {model.score(X_train_scaled, encoded_y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, encoded_y_test)}")

from sklearn.metrics import classification_report

predictions = model.predict(X_test_scaled)
print(classification_report(encoded_y_test, predictions, 
                            target_names = ["confirmed", "false positive", "candidate"]))

print(f"First 10 Predictions: {label_encoder.inverse_transform(predictions[:10])}")
print(f"First 10 Actual labels: {label_encoder.inverse_transform(encoded_y_test[:10])}")

Training Data Score: 0.8037383177570093
Testing Data Score: 0.8032036613272311
                precision    recall  f1-score   support

     confirmed       0.62      0.48      0.54       411
false positive       0.63      0.73      0.68       484
     candidate       0.98      1.00      0.99       853

      accuracy                           0.80      1748
     macro avg       0.74      0.74      0.74      1748
  weighted avg       0.80      0.80      0.80      1748

First 10 Predictions: ['FALSE POSITIVE' 'CANDIDATE' 'FALSE POSITIVE' 'FALSE POSITIVE'
 'FALSE POSITIVE' 'CONFIRMED' 'CONFIRMED' 'CANDIDATE' 'CANDIDATE'
 'FALSE POSITIVE']
First 10 Actual labels: ['FALSE POSITIVE' 'CANDIDATE' 'FALSE POSITIVE' 'FALSE POSITIVE'
 'FALSE POSITIVE' 'CONFIRMED' 'CANDIDATE' 'CANDIDATE' 'CANDIDATE'
 'FALSE POSITIVE']


## Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [15]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
            
param_grid = {'solver': ['newton-cg', 'lbfgs', 'liblinear'], 
              'penalty': ['l2'], 
              'C': [100, 10, 1, .01, .001], 
              'max_iter': [500]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [16]:
# Train the model with GridSearch
grid.fit(X_train_scaled, encoded_y_train)

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV 1/5] END C=100, max_iter=500, penalty=l2, solver=newton-cg;, score=0.830 total time=   0.2s
[CV 2/5] END C=100, max_iter=500, penalty=l2, solver=newton-cg;, score=0.800 total time=   0.2s
[CV 3/5] END C=100, max_iter=500, penalty=l2, solver=newton-cg;, score=0.805 total time=   0.3s
[CV 4/5] END C=100, max_iter=500, penalty=l2, solver=newton-cg;, score=0.804 total time=   0.2s
[CV 5/5] END C=100, max_iter=500, penalty=l2, solver=newton-cg;, score=0.815 total time=   0.2s
[CV 1/5] END C=100, max_iter=500, penalty=l2, solver=lbfgs;, score=0.830 total time=   1.0s
[CV 2/5] END C=100, max_iter=500, penalty=l2, solver=lbfgs;, score=0.800 total time=   0.6s
[CV 3/5] END C=100, max_iter=500, penalty=l2, solver=lbfgs;, score=0.805 total time=   0.6s
[CV 4/5] END C=100, max_iter=500, penalty=l2, solver=lbfgs;, score=0.804 total time=   0.7s
[CV 5/5] END C=100, max_iter=500, penalty=l2, solver=lbfgs;, score=0.815 total time=   0.8s

GridSearchCV(estimator=LogisticRegression(max_iter=400),
             param_grid={'C': [100, 10, 1, 0.01, 0.001], 'max_iter': [500],
                         'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear']},
             verbose=3)

In [17]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 100, 'max_iter': 500, 'penalty': 'l2', 'solver': 'newton-cg'}
0.8107949046347303


In [18]:
# Make predictions with hypertuned model
predictions = grid.predict(X_test_scaled)
print(classification_report(encoded_y_test, predictions, 
                            target_names = ["confirmed", "false positive", "candidate"]))

                precision    recall  f1-score   support

     confirmed       0.61      0.55      0.58       411
false positive       0.65      0.69      0.67       484
     candidate       0.98      1.00      0.99       853

      accuracy                           0.81      1748
     macro avg       0.75      0.74      0.74      1748
  weighted avg       0.80      0.81      0.80      1748



# Save the Model

In [19]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
#import joblib
#filename = 'LogisticRegression.sav'
#joblib.dump(model, filename)